In [2]:
!python -V

Python 3.9.6


In [3]:
import pandas as pd
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/aliaksei_zaleski/Documents/projects/mlops-practice/artifacts/1', creation_time=1766954204001, experiment_id='1', last_update_time=1766954204001, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
DATA_PATH = PROJECT_ROOT / "data" 
MODELS_PATH = PROJECT_ROOT / "models"

In [9]:
df_train = read_dataframe(DATA_PATH / "green_tripdata_2021-01.parquet")
df_val = read_dataframe(DATA_PATH / "green_tripdata_2021-02.parquet")

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715207732648

In [18]:
with open(MODELS_PATH / "lin_reg.bin", 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
with mlflow.start_run():

    mlflow.set_tag("developer", "alex")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path=MODELS_PATH / 'lin_reg.bin', artifact_path="models_pickle")

🏃 View run fun-skunk-177 at: http://localhost:5001/#/experiments/1/runs/b2384ac7a64b47f3a7ea9fc5ecfc6e37
🧪 View experiment at: http://localhost:5001/#/experiments/1


In [15]:
import xgboost as xgb

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, 'validation')],
            early_stopping_rounds=25,
            verbose_eval=False

        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

🏃 View run redolent-bass-535 at: http://localhost:5001/#/experiments/1/runs/994c4596b8874887b9e69005908538ce

🧪 View experiment at: http://localhost:5001/#/experiments/1

🏃 View run nosy-shrike-265 at: http://localhost:5001/#/experiments/1/runs/a4868b1a30904d359971b15f4b704f55

🧪 View experiment at: http://localhost:5001/#/experiments/1                   

🏃 View run vaunted-lynx-940 at: http://localhost:5001/#/experiments/1/runs/3eda3525e3c64c7abd457712610e1c7d

🧪 View experiment at: http://localhost:5001/#/experiments/1                   

🏃 View run rumbling-mule-373 at: http://localhost:5001/#/experiments/1/runs/80784e16e86b4fe481560bef6cf5c06e

🧪 View experiment at: http://localhost:5001/#/experiments/1                   

🏃 View run judicious-mule-347 at: http://localhost:5001/#/experiments/1/runs/f191ba423a2b493192909699aa8f62d0

🧪 View experiment at: http://localhost:5001/#/experiments/1                   

🏃 View run treasured-mole-193 at: http://localhost:5001/#/experiments/1

In [25]:
mlflow.xgboost.autolog(disable=True)

In [18]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 24
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=500,
        evals=[(valid, 'validation')],
        early_stopping_rounds=25,
        verbose_eval=False

    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open(MODELS_PATH / "vectorizer.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(MODELS_PATH / "vectorizer.b", artifact_path="preprocessor")
    mlflow.set_tag("model", "xgboost")
    mlflow.xgboost.log_model(booster, name="models_mlflow")

2025/12/28 23:01:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run capricious-koi-483 at: http://localhost:5001/#/experiments/1/runs/8e0f7ac6440b49409ce03a2b25654119
🧪 View experiment at: http://localhost:5001/#/experiments/1
